In [1]:
import sys
import os

# Assuming the package is one directory above the current working directory
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)

from diffUV import dyned_eul, dyn_body, kin, dyned_quat
from diffUV.utils.symbols import *
from diffUV.utils.quaternion_ops import euler2q, q2euler
from casadi import *
from blue_rov import Params as ps
import numpy as np
import os
from scipy.spatial.transform import Rotation as R

In [2]:
r_g = vertcat(x_g, y_g, z_g) # center of gravity wrt body origin
r_b = vertcat(x_b, y_b, z_b) # center of buoyancy wrt body origin 
I_o = vertcat(I_x, I_y, I_z,I_xz) # rigid body inertia wrt body origin

decoupled_added_m = vertcat(X_du, Y_dv, Z_dw, K_dp, M_dq, N_dr) # added mass in diagonals
coupled_added_m =  vertcat(X_dq, Y_dp, M_du, K_dv) # effective added mass in non diagonals

linear_dc = vertcat(X_u, Y_v, Z_w, K_p,  M_q, N_r) # linear damping coefficients
quadratic_dc = vertcat(X_uu, Y_vv, Z_ww, K_pp, M_qq, N_rr) # quadratic damping coefficients

n0 = vertcat(n, dn) # state variables wrt ned
v_r # velocity state variables wrt body

SX([(u-v_c_0), (v-v_c_1), (w-v_c_2), (p-v_c_3), (q-v_c_4), (r-v_c_5)])

In [3]:
# body representaion
uv_body = dyn_body()

# ned representaion
uv_ned = dyned_eul()

# quaternion representation
uv_quat = dyned_quat()

In [4]:
coupled_added_m

SX([X_dq, Y_dp, M_du, K_dv])

<h1>System Inertia Matrix example</h1>

In [5]:
inertia_mat = uv_body.body_inertia_matrix()
M_func = Function('M_b', [m, I_o, z_g, decoupled_added_m, coupled_added_m], [inertia_mat]) # for numerical & symbolic use

In [8]:
# example usage of inertia mat
M_mat = np.array(M_func(ps.m, ps.Io, ps.rg[2], ps.added_m, ps.coupl_added_m))
M_mat

array([[17.  ,  0.  ,  0.  ,  0.  ,  0.23,  0.  ],
       [ 0.  , 24.2 ,  0.  , -0.23,  0.  ,  0.  ],
       [ 0.  ,  0.  , 26.07,  0.  ,  0.  ,  0.  ],
       [ 0.  , -0.23,  0.  ,  0.28,  0.  ,  0.  ],
       [ 0.23,  0.  ,  0.  ,  0.  ,  0.28,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.28]])

In [9]:
M_mat.T == M_mat

array([[ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True]])

In [10]:
inertia_ned_mat = uv_ned.ned_euler_inertia_matrix()
Onb = vertcat(thet, phi, psi)
M_ned_func = Function('M_ned', [m, I_o, z_g, Onb, decoupled_added_m, coupled_added_m], [inertia_ned_mat]) # for numerical & symbolic use

In [11]:
# example usage of inertia mat in ned
Onb_v = np.array([0.13,0.17,0.1])
np.array(M_ned_func(ps.m, ps.Io, ps.rg[2], Onb_v,  ps.added_m, ps.coupl_added_m))

array([[ 1.72303360e+01, -7.45090577e-01,  1.18405662e+00,
         1.76115723e-02,  2.23648786e-01,  3.57838329e-02],
       [-7.45090577e-01,  2.41747103e+01, -1.91929773e-01,
        -2.26055609e-01,  2.24397276e-02,  3.31239544e-02],
       [ 1.18405662e+00, -1.91929773e-01,  2.58649536e+01,
        -3.85835972e-02, -2.93860503e-02,  0.00000000e+00],
       [ 1.76115723e-02, -2.26055609e-01, -3.85835972e-02,
         2.80000000e-01,  0.00000000e+00, -3.62975599e-02],
       [ 2.23648786e-01,  2.24397276e-02, -2.93860503e-02,
         0.00000000e+00,  2.80000000e-01,  0.00000000e+00],
       [ 3.57838329e-02,  3.31239544e-02,  0.00000000e+00,
        -3.62975599e-02,  0.00000000e+00,  2.80000000e-01]])

<h1>Coriolis Matrix example</h1>

In [14]:
coriolis_mat = uv_body.body_coriolis_centripetal_matrix()
C_func = Function('C_b', [m, I_o, z_g, decoupled_added_m, coupled_added_m, x_nb, v_c], [coriolis_mat]) # for numerical & symbolic use

In [15]:
C = np.array(C_func(ps.m, ps.Io, ps.rg[2], ps.added_m, ps.coupl_added_m, np.array([0.1 ,0.2,0.1,0.1,0.1,0.5]), 
                    np.array([0.0 ,0.0,0.0,0.0,0.0,0.0])))

In [16]:
-C.T == C

array([[ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True]])